# Prepare data

In [22]:
import pandas as pd
columns = ['game id', 'move id', 'player id', 'column', 'winner']
data = pd.read_csv('c4-50k.csv', header=None, names=columns)

data.head(3)

,game id,move id,player id,column,winner
0,11,0,-1,4,1
1,11,1,1,2,1
2,11,2,-1,4,1


In [23]:
print(len(data.loc[(data['move id'] == 0) & (data['column'] == 0)]))
print(len(data.loc[(data['move id'] == 0) & (data['column'] == 1)]))
print(len(data.loc[(data['move id'] == 0) & (data['column'] == 2)]))
print(len(data.loc[(data['move id'] == 0) & (data['column'] == 3)]))
print(len(data.loc[(data['move id'] == 0) & (data['column'] == 4)]))
print(len(data.loc[(data['move id'] == 0) & (data['column'] == 5)]))
print(len(data.loc[(data['move id'] == 0) & (data['column'] == 6)]))

2642
4068
6167
12606
9274
8211
7032


In [24]:
column_names = []
for i in range(6*7):
    column_names.append("current_" + str(i))
column_names.append("player")
column_names.append("nextmove column")
board_df = pd.DataFrame(columns=column_names)
board_df.describe()

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,current_34,current_35,current_36,current_37,current_38,current_39,current_40,current_41,player,nextmove column
count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
import copy
def generate_boards():
    for game_id in data['game id'].unique():
        current_board = [[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]]
        for move_id in data.loc[data['game id'] == game_id]['move id'].unique():
            next_board = copy.deepcopy(current_board)
            
            # update next board
            column = data.loc[(data['game id'] == game_id) & (data['move id'] == move_id)]['column'].unique()[0].item()
            player = data.loc[(data['game id'] == game_id) & (data['move id'] == move_id)]['player id'].unique()[0].item()
            winner = data.loc[data['game id'] == game_id]['winner'].unique()[0].item()
            
            for row in range(6):
                if next_board[row][column] == 0:
                    next_board[row][column] = player
                    break
            
            # if next board move comes from a winning player keep data
            if player == winner:
                full_board = copy.deepcopy(current_board)
                full_board.append([player])
                full_board.append([column])
                flat_board = [item for sublist in full_board for item in sublist]
                
                board_df.loc[len(board_df)] = flat_board
            
            # set the new move as the next current move
            current_board = copy.deepcopy(next_board)

In [26]:
generate_boards()

In [27]:
board_df.head(5)

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,current_34,current_35,current_36,current_37,current_38,current_39,current_40,current_41,player,nextmove column
0,0,0,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
1,0,0,1,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
2,0,0,1,1,-1,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,1,1,-1,0,-1,0,0,-1,...,0,0,0,0,0,0,0,0,1,1
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,3


In [28]:
board_df.to_csv('board_data.csv', index=False)

In [29]:
import pandas as pd
import numpy as np

data = pd.read_csv('board_data.csv')
data = data.groupby(data.columns.tolist(),as_index=False).size()
data['total_0'] = np.where(data['nextmove column'] == 0, data['size'], 0)
data['total_1'] = np.where(data['nextmove column'] == 1, data['size'], 0)
data['total_2'] = np.where(data['nextmove column'] == 2, data['size'], 0)
data['total_3'] = np.where(data['nextmove column'] == 3, data['size'], 0)
data['total_4'] = np.where(data['nextmove column'] == 4, data['size'], 0)
data['total_5'] = np.where(data['nextmove column'] == 5, data['size'], 0)
data['total_6'] = np.where(data['nextmove column'] == 6, data['size'], 0)
data.head(3)

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,player,nextmove column,size,total_0,total_1,total_2,total_3,total_4,total_5,total_6
0,-1,-1,-1,0,-1,1,-1,0,0,0,...,-1,3,1,0,0,0,1,0,0,0
1,-1,-1,-1,0,-1,1,0,0,1,0,...,-1,3,1,0,0,0,1,0,0,0
2,-1,-1,-1,0,-1,1,1,-1,0,0,...,-1,3,1,0,0,0,1,0,0,0


In [30]:
data = data.drop(columns=["nextmove column", "size"])

In [31]:
column_names = []
for i in range(6*7):
    column_names.append("current_" + str(i))
column_names.append("player")

data = data.groupby(column_names,as_index=False).sum()
data.loc[data['total_0'] > 5].head(3)

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,current_40,current_41,player,total_0,total_1,total_2,total_3,total_4,total_5,total_6
23362,-1,0,0,0,0,0,0,0,0,0,...,0,0,1,31,46,91,214,106,105,60
23372,-1,0,0,0,0,0,1,0,0,0,...,0,0,-1,12,9,8,33,16,14,18
23418,-1,0,0,0,0,1,0,0,0,0,...,0,0,-1,8,5,21,37,27,13,19


In [32]:
data['total'] = data[['total_0', 'total_1', 'total_2', 'total_3', 'total_4', 'total_5', 'total_6']].sum(axis=1)
data.loc[data['total'] > 5].head(3)

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,current_41,player,total_0,total_1,total_2,total_3,total_4,total_5,total_6,total
105,-1,-1,-1,0,1,0,1,0,0,0,...,0,1,0,0,0,7,0,0,0,7
2914,-1,-1,-1,1,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,6,6
2988,-1,-1,0,-1,0,1,0,0,0,0,...,0,-1,0,0,4,0,0,2,0,6


In [33]:
data['total_0'] = data['total_0'].div(data['total'])
data['total_1'] = data['total_1'].div(data['total'])
data['total_2'] = data['total_2'].div(data['total'])
data['total_3'] = data['total_3'].div(data['total'])
data['total_4'] = data['total_4'].div(data['total'])
data['total_5'] = data['total_5'].div(data['total'])
data['total_6'] = data['total_6'].div(data['total'])
data.loc[data['total'] > 5].head(10)

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,current_41,player,total_0,total_1,total_2,total_3,total_4,total_5,total_6,total
105,-1,-1,-1,0,1,0,1,0,0,0,...,0,1,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,7
2914,-1,-1,-1,1,1,1,0,0,0,0,...,0,1,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,6
2988,-1,-1,0,-1,0,1,0,0,0,0,...,0,-1,0.0,0.0,0.666667,0.000000,0.000000,0.333333,0.000000,6
2990,-1,-1,0,-1,0,1,0,0,0,0,...,0,1,0.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,11
3156,-1,-1,0,-1,1,0,0,0,0,0,...,0,1,0.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,8
3166,-1,-1,0,-1,1,0,0,0,0,0,...,0,1,0.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,9
3299,-1,-1,0,-1,1,1,0,0,0,0,...,0,-1,0.0,0.0,0.833333,0.166667,0.000000,0.000000,0.000000,6
3306,-1,-1,0,-1,1,1,0,0,0,0,...,0,1,0.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,7
3461,-1,-1,0,0,0,0,1,0,0,0,...,0,1,0.0,0.0,0.181818,0.454545,0.181818,0.000000,0.181818,11
3464,-1,-1,0,0,0,0,1,0,0,0,...,0,1,0.0,0.0,0.166667,0.000000,0.333333,0.000000,0.500000,6


In [34]:
# Most common winning column is the choosen column
m = np.zeros_like(data[['total_0', 'total_1', 'total_2', 'total_3', 'total_4', 'total_5', 'total_6']].values)
m[np.arange(len(data[['total_0', 'total_1', 'total_2', 'total_3', 'total_4', 'total_5', 'total_6']])), data[['total_0', 'total_1', 'total_2', 'total_3', 'total_4', 'total_5', 'total_6']].values.argmax(1)] = 1
df1 = pd.DataFrame(m, columns = [['total_0', 'total_1', 'total_2', 'total_3', 'total_4', 'total_5', 'total_6']]).astype(int)
data['total_0'] = df1['total_0']
data['total_1'] = df1['total_1']
data['total_2'] = df1['total_2']
data['total_3'] = df1['total_3']
data['total_4'] = df1['total_4']
data['total_5'] = df1['total_5']
data['total_6'] = df1['total_6']
data.loc[data['total'] > 5].head(10)

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,current_41,player,total_0,total_1,total_2,total_3,total_4,total_5,total_6,total
105,-1,-1,-1,0,1,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,7
2914,-1,-1,-1,1,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,6
2988,-1,-1,0,-1,0,1,0,0,0,0,...,0,-1,0,0,1,0,0,0,0,6
2990,-1,-1,0,-1,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,11
3156,-1,-1,0,-1,1,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,8
3166,-1,-1,0,-1,1,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,9
3299,-1,-1,0,-1,1,1,0,0,0,0,...,0,-1,0,0,1,0,0,0,0,6
3306,-1,-1,0,-1,1,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,7
3461,-1,-1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,11
3464,-1,-1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,6


In [35]:
data = data.drop(columns=["total"])
data.to_csv('board_data2.csv', index=False)

# Prepare model

In [1]:
import pandas as pd

data = pd.read_csv('board_data2.csv')
data.head(3)

,current_0,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,...,current_40,current_41,player,total_0,total_1,total_2,total_3,total_4,total_5,total_6
0,-1,-1,-1,0,-1,1,-1,0,0,0,...,0,0,-1,0,0,0,1,0,0,0
1,-1,-1,-1,0,-1,1,0,0,1,0,...,0,0,-1,0,0,0,1,0,0,0
2,-1,-1,-1,0,-1,1,1,-1,0,0,...,0,0,-1,0,0,0,1,0,0,0


In [2]:
from sklearn.model_selection import train_test_split
from numpy.random import seed
seed(1746890)
from tensorflow.random import set_seed
set_seed(1746890)

train, test = train_test_split(data, test_size=0.2, random_state=1746890, shuffle=True)

column_names = []
for i in range(6*7):
    column_names.append("current_" + str(i))
column_names.append("player")

X_train = train.drop(columns=["total_0", "total_1", "total_2", "total_3", "total_4", "total_5", "total_6"], axis=1)
X_test = test.drop(columns=["total_0", "total_1", "total_2", "total_3", "total_4", "total_5", "total_6"], axis=1)
Y_train = train.drop(columns=column_names, axis=1)
Y_test = test.drop(columns=column_names, axis=1)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(184166, 43)
(46042, 43)
(184166, 7)
(46042, 7)


In [3]:
Y_train.head(3)

,total_0,total_1,total_2,total_3,total_4,total_5,total_6
21612,0,0,0,0,0,0,1
224989,0,1,0,0,0,0,0
170878,0,0,0,1,0,0,0


In [4]:
X_train = X_train.values
X_test = X_test.values
Y_train = Y_train.values
Y_test = Y_test.values
print(X_train)

[[-1  0 -1 ...  0  0  1]
 [ 1  1  0 ...  0  0 -1]
 [ 1 -1 -1 ...  0  0  1]
 ...
 [ 0  1 -1 ...  0  0 -1]
 [ 0  0 -1 ...  0  0 -1]
 [ 0 -1  0 ...  0  0  1]]


In [5]:
from tensorflow import keras

inputs = keras.Input(shape=X_train.shape[1])
hidden_layer = keras.layers.Dense(128, activation="sigmoid")(inputs)
hidden_layer = keras.layers.Dense(128, activation="sigmoid")(hidden_layer)
output_layer = keras.layers.Dense(7, activation="softmax")(hidden_layer)

model = keras.Model(inputs=inputs, outputs=output_layer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 43)]              0         
                                                                 
 dense (Dense)               (None, 128)               5632      
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 7)                 903       
                                                                 
Total params: 23,047
Trainable params: 23,047
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='adam', loss=keras.losses.CategoricalCrossentropy(), metrics=['Accuracy'])

In [7]:
history = model.fit(X_train, Y_train, epochs=100, validation_data=[X_test, Y_test])

Epoch 1/100
5756/5756 [==============================] - 19s 3ms/step - loss: 1.9079 - Accuracy: 0.1816 - val_loss: 1.9034 - val_Accuracy: 0.1845
Epoch 2/100
5756/5756 [==============================] - 19s 3ms/step - loss: 1.8861 - Accuracy: 0.2019 - val_loss: 1.8549 - val_Accuracy: 0.2292
Epoch 3/100
5756/5756 [==============================] - 19s 3ms/step - loss: 1.8061 - Accuracy: 0.2768 - val_loss: 1.7658 - val_Accuracy: 0.3029
Epoch 4/100
5756/5756 [==============================] - 18s 3ms/step - loss: 1.7364 - Accuracy: 0.3205 - val_loss: 1.7210 - val_Accuracy: 0.3212
Epoch 5/100
5756/5756 [==============================] - 18s 3ms/step - loss: 1.6951 - Accuracy: 0.3359 - val_loss: 1.6748 - val_Accuracy: 0.3396
Epoch 6/100
5756/5756 [==============================] - 19s 3ms/step - loss: 1.6503 - Accuracy: 0.3551 - val_loss: 1.6364 - val_Accuracy: 0.3584
Epoch 7/100
5756/5756 [==============================] - 19s 3ms/step - loss: 1.6154 - Accuracy: 0.3701 - val_loss: 1.6119 -

Epoch 57/100
5756/5756 [==============================] - 18s 3ms/step - loss: 1.3955 - Accuracy: 0.4685 - val_loss: 1.5245 - val_Accuracy: 0.4200
Epoch 58/100
5756/5756 [==============================] - 19s 3ms/step - loss: 1.3942 - Accuracy: 0.4690 - val_loss: 1.5231 - val_Accuracy: 0.4214
Epoch 59/100
5756/5756 [==============================] - 19s 3ms/step - loss: 1.3932 - Accuracy: 0.4702 - val_loss: 1.5291 - val_Accuracy: 0.4206
Epoch 60/100
5756/5756 [==============================] - 19s 3ms/step - loss: 1.3919 - Accuracy: 0.4703 - val_loss: 1.5278 - val_Accuracy: 0.4208
Epoch 61/100
5756/5756 [==============================] - 19s 3ms/step - loss: 1.3906 - Accuracy: 0.4701 - val_loss: 1.5254 - val_Accuracy: 0.4211
Epoch 62/100
5756/5756 [==============================] - 20s 3ms/step - loss: 1.3900 - Accuracy: 0.4711 - val_loss: 1.5297 - val_Accuracy: 0.4188
Epoch 63/100
5756/5756 [==============================] - 20s 3ms/step - loss: 1.3887 - Accuracy: 0.4707 - val_loss: 1

In [10]:
model.save('model3')

INFO:tensorflow:Assets written to: model3\assets


In [11]:
from tensorflow import keras
pretrained_model = keras.models.load_model('model3')

In [12]:
board = [[0,1,1,1,-1,-1,0],[0,0,-1,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0], [-1]]
flat_board = [item for sublist in board for item in sublist]
pred = model.predict([flat_board])
print(pred)

[[9.52892065e-01 3.17513739e-04 1.12183085e-02 2.88053863e-02
  4.77140537e-03 1.49733853e-03 4.98045119e-04]]


In [13]:
import numpy as np
pred_column = np.array(pred[0]).argmax()
pred_chance = pred[0][pred_column] * 100
print('column {} with {:.2f}%'.format(pred_column, pred_chance))

column 0 with 95.29%
